In [1]:
import pandas as pd
import numpy as np

# read data from xls file 
df = pd.read_csv('features.csv', header = None, na_values = '?', index_col = None)

# filenames = ['features2.csv', 'features3.csv', 'features4.csv', 'features5.csv', 'features6.csv','test1.csv','test2.csv']
# for filename in filenames:
#     df1 = pd.read_csv(filename, header = None, na_values = '0', index_col = None)
#     df = pd.concat([df,df1], ignore_index = True)
data_column = np.shape(df)[1]
data_row = np.shape(df)[0]
print(np.shape(df))
df.head(5)


(2618, 1787)


,0,1,2,3,4,5,6,7,8,9,...,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786
0,-94.654073,-85.509316,-89.514151,-75.158692,-54.101195,16.318827,111.716281,114.988910,117.105902,100.701295,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
1,84.600350,44.900890,3.827597,5.621762,0.194946,6.518361,5.823970,10.533665,11.121295,9.023582,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
2,-85.126966,-89.579010,-41.984810,49.378000,108.657153,109.867511,119.388176,116.645629,112.809038,109.047258,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
3,-63.233798,20.978620,99.049608,95.757327,78.557140,84.435719,83.664807,68.762827,67.117229,62.209639,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
4,87.972568,26.667942,0.224216,8.267901,22.654129,31.920317,18.691901,1.916780,5.505791,9.660367,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0


In [2]:
# get the value 
y_raw = np.array(df[data_column-2])

# list of 1D-features in the order of MFCC, Energy, Pitch
X_raw = np.array(pd.DataFrame(df, columns = df.columns[0:(data_column-2)]))

print(np.shape(X_raw))
print(np.shape(y_raw))
# print(X_raw)
# print(y_raw)


(2618, 1785)
(2618,)


In [3]:
# one hot coding 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import Counter

y_reshape = y_raw.reshape(-1, 1)
encoder = preprocessing.OneHotEncoder(sparse=False)
yhot = encoder.fit_transform(y_reshape)
print(yhot)
print(yhot.shape)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
(2618, 7)


In [4]:
# preprocessing the data
# normalize
ydim = 21
xdim = 85

X = preprocessing.scale(X_raw)
# print(X.shape)
# scaler = preprocessing.standardScaler().fit(X_raw)
# X = scaler.transform(X_re)

(2618, 1785)


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
test_rate = 0.2
Xtr, Xts, ytr, yts = train_test_split(X, y_raw, test_size=test_rate, random_state=0)
print(Xtr.shape)
print(ytr.shape)
print(Xts.shape)
print(yts.shape)

cnt = Counter(ytr)
print("Number of files in each category in TRAIN set:")
for k in sorted(cnt.keys()):
    print(cnt[k])

(2094, 1785)
(2094,)
(524, 1785)
(524,)
Number of files in each category in TRAIN set:
318
240
308
305
310
300
313


In [ ]:
## use SVM model to predict class

In [6]:
from sklearn import svm
svc = svm.SVC(probability=False,  kernel="rbf", C=2.8, gamma=.0073,verbose=10)
svc.fit(Xtr,ytr)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0073, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [7]:
yhat_ts = svc.predict(Xts)
acc = np.mean(yhat_ts == yts)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.211832


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import precision_recall_fscore_support
# from random import shuffle

# def train_classifier(xtr, ytr):
#     print(xtr.shape)
#     print(ytr.shape)
#     dataset = np.hstack([xtr,xtr])
#     shuffle(dataset)
#     xtr = dataset[0:136]
#     ytr = dataset[137]
#     clf = RandomForestClassifier(n_estimators=1000, max_depth = 100000, n_jobs = -1, oob_score = True, min_samples_split = 10)
#     clf.fit(xtr, ytr)
#     return clf


# def test_classifier(classifier, xts, yts):
#     y_pred = classifier.predict(xts)
#     precision, recall, f1_score, _ = precision_recall_fscore_support(y_ts, y_pred)
#     return [precision, recall, f1_score]



In [ ]:
# clf = train_classifier(xtr,ytr)
# [precision, recall, f1_score] = test_classifier(clf,xts,yts)

In [8]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input, Convolution2D, MaxPooling2D, Activation, merge
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [9]:
import keras.backend as K
K.clear_session()

In [48]:
# try to use pre-trained deep learning network vgg16
Xtr = Xtr.reshape(Xtr.shape[0], ydim, xdim, 1)
Xts = Xts.reshape(Xts.shape[0], ydim, xdim, 1)
print(Xtr.shape)
print(Xts.shape)
Xtr_1 = []
Xts_1 = []
Xtr_1 = Xtr.repeat(3, axis=1)
Xts_1 = Xts.repeat(3, axis=1)
Xtr_1 = Xtr_1.repeat(3, axis=3)
Xts_1 = Xts_1.repeat(3, axis=3)
print(Xtr_1.shape)
print(Xts_1.shape)

pre_trained = 'vgg16'

# Load appropriate packages
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import decode_predictions, preprocess_input    

input_shape = (ydim*3,xdim,3)
base_model = applications.VGG16(weights='imagenet', include_top = False, input_shape = input_shape)

(2094, 21, 85, 1)
(524, 21, 85, 1)
(2094, 63, 85, 3)
(524, 63, 85, 3)


In [49]:
model = Sequential()

for layers in base_model.layers:
    model.add(layers)
    
for layers in model.layers:
    layers.trainable = False
    
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 63, 85, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 63, 85, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 63, 85, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 31, 42, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 31, 42, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 31, 42, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 15, 21, 128)       0         
__________

In [50]:
print(Xtr_1.shape)
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow(
                        Xtr_1,ytr,
                        batch_size=batch_size)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(
                        Xts_1,yts, 
                        batch_size=batch_size)

(2094, 63, 85, 3)


In [51]:
opt = optimizers.Adam(lr=0.001) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [52]:
train_size = train_generator.n
test_size = test_generator.n
# print(train_size)
# print(test_size)
steps_per_epoch =  train_size // batch_size
validation_steps =  test_size // batch_size

In [53]:
nepochs = 5  # Number of epochs

# Call the fit function
model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nepochs,
    validation_data=test_generator,
    validation_steps=validation_steps)

Epoch 1/5
65/65 [==============================] - 156s - loss: -27.1110 - acc: 0.1149 - val_loss: -34.0644 - val_acc: 0.0996
Epoch 2/5
65/65 [==============================] - 154s - loss: -32.7714 - acc: 0.1147 - val_loss: -33.6345 - val_acc: 0.0874
Epoch 3/5
65/65 [==============================] - 153s - loss: -32.6977 - acc: 0.1157 - val_loss: -34.2826 - val_acc: 0.1016
Epoch 4/5
65/65 [==============================] - 161s - loss: -33.0473 - acc: 0.1128 - val_loss: -33.3753 - val_acc: 0.1220
Epoch 5/5
65/65 [==============================] - 156s - loss: -32.4306 - acc: 0.1149 - val_loss: -34.5094 - val_acc: 0.0955


In [ ]:
## another method use self defined CNN model

In [88]:
#kares package use Tensorflow as backend
# Xtr = Xtr.reshape(Xtr.shape[0], ydim, xdim, 1)
# Xts = Xts.reshape(Xts.shape[0], ydim, xdim, 1)
test_rate = 0.2
Xtr, Xts, ytr, yts = train_test_split(X, y_raw, test_size=test_rate, random_state=0)
print(Xtr.shape)
print(ytr)
print(Xts.shape)
print(yts.shape)
Xtr = Xtr.reshape(Xtr.shape[0], ydim, xdim, 1)
Xts = Xts.reshape(Xts.shape[0], ydim, xdim, 1)
ytr_reshape = ytr.reshape(-1, 1)
encoder = preprocessing.OneHotEncoder(sparse=False)
ytr_hot = encoder.fit_transform(ytr_reshape)

# cnt = Counter(Xtr)
# print("Number of files in each category in TRAIN set:")
# for k in sorted(cnt.keys()):
#     print(cnt[k])
# print(Xtr.shape)
# print(Xts.shape)
# in_shape = Xtr.shape[1:]

(2094, 1785)
[2 5 6 ..., 4 1 6]
(524, 1785)
(524,)


In [89]:
np.random.seed(0) 

model = Sequential()

conv_filters = 16   # number of convolution filters (= CNN depth)
#conv_filters = 32   # number of convolution filters (= CNN depth)


# Layer 1
model.add(Convolution2D(conv_filters, 3, 3, input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2, 2))) 
#model.add(Dropout(0.25)) 

# Layer 2
model.add(Convolution2D(conv_filters, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2))) 

# After Convolution, we have a 16*x*y matrix output
# In order to feed this to a Full(Dense) layer, we need to flatten all data
# Note: Keras does automatic shape inference, i.e. it knows how many (flat) input units the next layer will need,
# so no parameter is needed for the Flatten() layer.
model.add(Flatten()) 

# Full layer
model.add(Dense(200, activation='sigmoid')) 
#model.add(Dense(256, activation='sigmoid')) 
model.add(Dropout(0.25))

# Output layer
# For binary/2-class problems use ONE sigmoid unit, for multi-class/multi-label problems use n output units 
# activation should be 'softmax' for multi-class / single-label output, 'sigmoid' for binary or multi-label tasks
model.add(Dense(7,activation='softmax'))

/Users/irenecai/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(21, 85, 1...)`
  # Remove the CWD from sys.path while we load stuff.
/Users/irenecai/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3))`
  from ipykernel import kernelapp as app


In [90]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 19, 83, 16)        160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 9, 41, 16)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 7, 39, 16)         2320      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 3, 19, 16)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 912)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 200)               182600    
_________________________________________________________________
dropout_8 (Dropout)          (None, 200)               0         
__________

In [91]:
# Define a loss function 
loss = 'categorical_crossentropy' 
learn_rate = 0.5
# Optimizer = Stochastic Gradient Descent
optimizer = optimizers.SGD(lr=learn_rate)

# Compiling the model
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [92]:
history = None

In [93]:
# TRAINING the model

# YOU MAY RUN THIS CELL MULTIPLE TIMES TO CONTINUE TO TRAIN THE MODEL FURTHER

# for how many epochs (iterations) to train
epochs = 15

# for training we need the "1 hot encoded" numeric classes of the ground truth
# History = model.fit(train_set, train_classes_1hot, batch_size=32, nb_epoch=epochs)
validation_percent = 0.1
History = model.fit(Xtr, ytr_hot, validation_split=validation_percent, batch_size=32, epochs=epochs)

# we keep the history of accuracies on training set
# we append this to previous history in case we execute this cell multiple times
if history is None:
    history = History.history
else:
    for key in History.history.keys():
        history[key].extend(History.history[key])

Train on 1884 samples, validate on 210 samples
Epoch 1/15
1884/1884 [==============================] - 1s - loss: 2.0115 - acc: 0.2489 - val_loss: 1.4210 - val_acc: 0.4571
Epoch 2/15
1884/1884 [==============================] - 1s - loss: 1.1437 - acc: 0.5817 - val_loss: 1.0210 - val_acc: 0.6238
Epoch 3/15
1884/1884 [==============================] - 1s - loss: 0.8679 - acc: 0.6842 - val_loss: 0.8560 - val_acc: 0.6667
Epoch 4/15
1884/1884 [==============================] - ETA: 0s - loss: 0.7152 - acc: 0.743 - 1s - loss: 0.7261 - acc: 0.7404 - val_loss: 0.8446 - val_acc: 0.7048
Epoch 5/15
1884/1884 [==============================] - 1s - loss: 0.6116 - acc: 0.7797 - val_loss: 0.7135 - val_acc: 0.7429
Epoch 6/15
1884/1884 [==============================] - 1s - loss: 0.5455 - acc: 0.8057 - val_loss: 0.8020 - val_acc: 0.7095
Epoch 7/15
1884/1884 [==============================] - 1s - loss: 0.4891 - acc: 0.8254 - val_loss: 0.7017 - val_acc: 0.7524
Epoch 8/15
1884/1884 [==================

In [94]:
test_pred = model.predict_classes(Xts)

288/524 [===============>..............] - ETA: 0s

In [97]:
test_pred.shape

(524,)

In [96]:
accuracy_score(yts, test_pred)

0.76717557251908397

In [ ]:
# print(xtr.shape)
# nin = xtr.shape[1]  # dimension of input data
# nh = 100     # number of hidden units
# nout = 8    # number of outputs = 10 since there are 10 classes
# model = Sequential()
# model.add(Dense(nh, input_shape=(nin,), activation='sigmoid', name='hidden'))
# model.add(Dense(nout, activation='softmax', name='output'))
# model.summary()

In [ ]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         # TODO:  Create two empty lists, self.loss and self.val_acc
#         self.loss = []
#         self.val_acc = []
 
#     def on_batch_end(self, batch, logs={}):
#         # TODO:  This is called at the end of each batch.  
#         # Add the loss in logs.get('loss') to the loss list
#         loss = logs.get('loss')
#         self.loss.append(loss)
        
#     def on_epoch_end(self, epoch, logs):
#         # TODO:  This is called at the end of each epoch.  
#         # Add the test accuracy in logs.get('loss') to the val_acc list
#         acc = logs.get('val_acc')
#         self.val_acc.append(acc)

# # Create an instance of the history callback
# history_cb = LossHistory()

In [ ]:
# from keras import optimizers
# opt = optimizers.Adam(lr=1e-5) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(optimizer=opt,
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# batch_size = 50
# model.fit(xtr, ytr, epochs=10, batch_size=batch_size, validation_data=(xts,yts), callbacks=[history_cb])